In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()
# this is my kaggle .json you can upload your kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pranshu29","key":"545552237d6ad0822fca7e106dd551fd"}'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# this cell is important to execute as the toy dataset in your drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Oct  2 08:08 kaggle.json


In [4]:
# Next, install the Kaggle API client.
!pip install -q kaggle
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
# List available datasets.
!kaggle datasets list

ref                                                       title                                               size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
dgomonov/new-york-city-airbnb-open-data                   New York City Airbnb Open Data                       2MB  2019-08-12 16:24:45          23056  
lakshyaag/india-trade-data                                India - Trade Data                                   1MB  2019-08-16 16:13:58          10969  
therohk/ireland-historical-news                           The Irish Times - Waxy-Wany News                    47MB  2019-08-24 15:36:54           2360  
dareenalharthi/jamalon-arabic-books-dataset               Jamalon Arabic Books Dataset                         1MB  2019-08-15 18:58:06            646  
rajeevw/ufcdata                                           UFC-Fight historical dat

In [5]:
!kaggle datasets download -d mfekadu/darpa-timit-acousticphonetic-continuous-speech
!unzip darpa-timit-acousticphonetic-continuous-speech.zip
!unzip data.zip
!pip install webrtcvad

 99% 818M/826M [00:18<00:00, 42.2MB/s]
100% 826M/826M [00:18<00:00, 47.6MB/s]
Archive:  darpa-timit-acousticphonetic-continuous-speech.zip
  inflating: TIMITDIC.DOC            
  inflating: PROMPTS.TXT             
  inflating: train_data.csv          
  inflating: test_data.csv           
  inflating: SPKRINFO.TXT            
  inflating: data.zip                
  inflating: SPKRSENT.TXT            
  inflating: README.DOC              
  inflating: TIMITDIC.TXT            
  inflating: PHONCODE.DOC            
  inflating: TESTSET.DOC             
Archive:  data.zip
   creating: TEST/
   creating: TEST/DR4/
   creating: TEST/DR4/MGMM0/
  inflating: TEST/DR4/MGMM0/SX139.WAV  
  inflating: TEST/DR4/MGMM0/SX139.WAV.wav  
  inflating: TEST/DR4/MGMM0/SX139.TXT  
  inflating: TEST/DR4/MGMM0/SI499.WRD  
  inflating: TEST/DR4/MGMM0/SX319.WRD  
  inflating: TEST/DR4/MGMM0/SX319.PHN  
  inflating: TEST/DR4/MGMM0/SI499.PHN  
  inflating: TEST/DR4/MGMM0/SI1129.WAV.wav  
  inflating: TEST/DR4/MG

In [0]:
dict1={"training": True,
"device": "cuda",
"unprocessed_data":'TRAIN/*/*/*.wav  ',
"train_path" : 'train_tisv',
"train_path_unprocessed": 'TRAIN/*/*/*.wav',
"test_path": 'test_tisv',
"test_path_unprocessed": 'TRAIN/*/*/*.wav',
"data_preprocessed":True ,
"sr": 16000,
"nfft": 512,
"window": 0.025 ,
"hop": 0.01 ,
"nmels": 40,
"tisv_frame": 180 , 
"hidden": 768 ,
"num_layer": 3 ,
"proj": 256 ,
"model_path": 'model' ,
"N" : 4 ,
"M" : 5 ,
"num_workers": 0 ,
"lr": 0.01 ,
"epochs": 450 ,
"log_interval": 30 ,
"log_file": 'speech_id_checkpoint/Stats',
"checkpoint_interval": 120 ,
"checkpoint_dir": 'speech_id_checkpoint',
"restore": False ,
"N" : 4 ,
"M" : 6 ,
"num_workers": 8,
"epochs_t": 10 }

In [0]:
import librosa
import numpy as np
import torch
import torch.autograd as grad
import torch.nn.functional as F
def get_centroids(embeddings):
    centroids = []
    for speaker in embeddings:
        centroid = 0
        for utterance in speaker:
            centroid = centroid + utterance
        centroid = centroid/len(speaker)
        centroids.append(centroid)
    centroids = torch.stack(centroids)
    return centroids

def get_centroid(embeddings, speaker_num, utterance_num):
    centroid = 0
    for utterance_id, utterance in enumerate(embeddings[speaker_num]):
        if utterance_id == utterance_num:
            continue
        centroid = centroid + utterance
    centroid = centroid/(len(embeddings[speaker_num])-1)
    return centroid

def get_cossim(embeddings, centroids):
    # Calculates cosine similarity matrix. Requires (N, M, feature) input
    cossim = torch.zeros(embeddings.size(0),embeddings.size(1),centroids.size(0))
    for speaker_num, speaker in enumerate(embeddings):
        for utterance_num, utterance in enumerate(speaker):
            for centroid_num, centroid in enumerate(centroids):
                if speaker_num == centroid_num:
                    centroid = get_centroid(embeddings, speaker_num, utterance_num)
                output = F.cosine_similarity(utterance,centroid,dim=0)+1e-6
                cossim[speaker_num][utterance_num][centroid_num] = output
    return cossim

def calc_loss(sim_matrix):
    # Calculates loss from (N, M, K) similarity matrix
    per_embedding_loss = torch.zeros(sim_matrix.size(0), sim_matrix.size(1))
    for j in range(len(sim_matrix)):
        for i in range(sim_matrix.size(1)):
            per_embedding_loss[j][i] = -(sim_matrix[j][i][j] - ((torch.exp(sim_matrix[j][i]).sum()+1e-6).log_()))
    loss = per_embedding_loss.sum()    
    return loss, per_embedding_loss

def normalize_0_1(values, max_value, min_value):
    normalized = np.clip((values - min_value) / (max_value - min_value), 0, 1)
    return normalized

def mfccs_and_spec(wav_file, wav_process = False, calc_mfccs=False, calc_mag_db=False):    
    sound_file, _ = librosa.core.load(wav_file, sr=hp.data.sr)
    window_length = int(hp.data.window*hp.data.sr)
    hop_length = int(hp.data.hop*hp.data.sr)
    duration = hp.data.tisv_frame * hp.data.hop + hp.data.window
    
    # Cut silence and fix length
    if wav_process == True:
        sound_file, index = librosa.effects.trim(sound_file, frame_length=window_length, hop_length=hop_length)
        length = int(hp.data.sr * duration)
        sound_file = librosa.util.fix_length(sound_file, length)
        
    spec = librosa.stft(sound_file, n_fft=hp.data.nfft, hop_length=hop_length, win_length=window_length)
    mag_spec = np.abs(spec)
    
    mel_basis = librosa.filters.mel(hp.data.sr, hp.data.nfft, n_mels=hp.data.nmels)
    mel_spec = np.dot(mel_basis, mag_spec)
    
    mag_db = librosa.amplitude_to_db(mag_spec)
    #db mel spectrogram
    mel_db = librosa.amplitude_to_db(mel_spec).T
    
    mfccs = None
    if calc_mfccs:
        mfccs = np.dot(librosa.filters.dct(40, mel_db.shape[0]), mel_db).T
    
    return mfccs, mel_db, mag_db

if __name__ == "__main__":
    w = grad.Variable(torch.tensor(1.0))
    b = grad.Variable(torch.tensor(0.0))
    embeddings = torch.tensor([[0,1,0],[0,0,1], [0,1,0], [0,1,0], [1,0,0], [1,0,0]]).to(torch.float).reshape(3,2,3)
    centroids = get_centroids(embeddings)
    cossim = get_cossim(embeddings, centroids)
    sim_matrix = w*cossim + b
    loss, per_embedding_loss = calc_loss(sim_matrix)

In [0]:
import collections
import contextlib
import numpy as np
import sys
import librosa
import wave

import webrtcvad
sr=dict1['sr']
def read_wave(path, sr):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    Assumes sample width == 2
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        pcm_data = wf.readframes(wf.getnframes())
    data, _ = librosa.load(path, sr)
    assert len(data.shape) == 1
    assert sr in (8000, 16000, 32000, 48000)
    return data, pcm_data
    
class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                start = ring_buffer[0][0].timestamp
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                triggered = False
                yield (start, frame.timestamp + frame.duration)
                ring_buffer.clear()
                voiced_frames = []
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield (start, frame.timestamp + frame.duration)

def VAD_chunk(aggressiveness, path):
    audio, byte_audio = read_wave(path, sr)
    vad = webrtcvad.Vad(int(aggressiveness))
    frames = frame_generator(20, byte_audio,sr)
    frames = list(frames)
    times = vad_collector(sr, 20, 200, vad, frames)
    speech_times = []
    speech_segs = []
    for i, time in enumerate(times):
        start = np.round(time[0],decimals=2)
        end = np.round(time[1],decimals=2)
        j = start
        while j + .4 < end:
            end_j = np.round(j+.4,decimals=2)
            speech_times.append((j, end_j))
            speech_segs.append(audio[int(j*sr):int(end_j*sr)])
            j = end_j
        else:
            speech_times.append((j, end))
            speech_segs.append(audio[int(j*sr):int(end*sr)])
    return speech_times, speech_segs
speech_times, speech_segs = VAD_chunk(3,"TRAIN/DR8/MRDM0/SI1044.WAV.wav")

In [0]:
import glob
import numpy as np
import os
import random
from random import shuffle
import torch
from torch.utils.data import Dataset

class SpeakerDatasetTIMIT(Dataset):
    def __init__(self,dict1):

        if dict1['training']:
            self.path = dict1['train_path_unprocessed']
            self.utterance_number = dict1['M']
        else:
            self.path = dict1['test_path_unprocessed']
            self.utterance_number = dict1['M']
        self.speakers = glob.glob(os.path.dirname(self.path))
        shuffle(self.speakers)
        
    def __len__(self):
        return len(self.speakers)

    def __getitem__(self, idx):
        
        speaker = self.speakers[idx]
        wav_files = glob.glob(speaker+'/*.WAV')
        shuffle(wav_files)
        wav_files = wav_files[0:self.utterance_number]
        
        mel_dbs = []
        for f in wav_files:
            _, mel_db, _ = mfccs_and_spec(f, wav_process = True)
            mel_dbs.append(mel_db)
        return torch.Tensor(mel_dbs)

class SpeakerDatasetTIMITPreprocessed(Dataset):
    
    def __init__(self,dict1, shuffle=True, utter_start=0):
        
        # data path
        if dict1['training']:
            self.path = dict1['train_path']
            self.utter_num = dict1['M']
        else:
            self.path = dict1['test_path']
            self.utter_num = dict1['M']
        self.file_list = os.listdir(self.path)
        self.shuffle=shuffle
        self.utter_start = utter_start
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        
        np_file_list = os.listdir(self.path)
        
        if self.shuffle:
            selected_file = random.sample(np_file_list, 1)[0]  # select random speaker
        else:
            selected_file = np_file_list[idx]               
        
        utters = np.load(os.path.join(self.path, selected_file))        # load utterance spectrogram of selected speaker
        if self.shuffle:
            utter_index = np.random.randint(0, utters.shape[0], self.utter_num)   # select M utterances per speaker
            utterance = utters[utter_index]       
        else:
            utterance = utters[self.utter_start: self.utter_start+self.utter_num] # utterances of a speaker [batch(M), n_mels, frames]

        utterance = utterance[:,:,:160]               # TODO implement variable length batch size

        utterance = torch.tensor(np.transpose(utterance, axes=(0,2,1)))     # transpose [batch, frames, n_mels]
        return utterance

In [0]:
import torch
import torch.nn as nn
class SpeechEmbedder(nn.Module):
    
    def __init__(self,dict1):
        super(SpeechEmbedder, self).__init__()    
        self.LSTM_stack = nn.LSTM(dict1['nmels'], dict1['hidden'], dict1['num_layer'], batch_first=True)
        for name, param in self.LSTM_stack.named_parameters():
          if 'bias' in name:
             nn.init.constant_(param, 0.0)
          elif 'weight' in name:
             nn.init.xavier_normal_(param)
        self.projection = nn.Linear(dict1['hidden'], dict1['proj'])
        
    def forward(self, x):
        x, _ = self.LSTM_stack(x.float()) #(batch, frames, n_mels)
        #only use last frame
        x = x[:,x.size(1)-1]
        x = self.projection(x.float())
        x = x / torch.norm(x, dim=1).unsqueeze(1)
        return x

class GE2ELoss(nn.Module):
    
    def __init__(self, device):
        super(GE2ELoss, self).__init__()
        self.w = nn.Parameter(torch.tensor(10.0).to(device), requires_grad=True)
        self.b = nn.Parameter(torch.tensor(-5.0).to(device), requires_grad=True)
        self.device = device
        
    def forward(self, embeddings):
        torch.clamp(self.w, 1e-6)
        centroids = get_centroids(embeddings)
        cossim = get_cossim(embeddings, centroids)
        sim_matrix = self.w*cossim.to(self.device) + self.b
        loss, _ = calc_loss(sim_matrix)
        return loss

In [11]:
import glob
import os
import librosa
import numpy as np
audio_path = glob.glob(os.path.dirname(dict1['unprocessed_data']))                                      
a=[]
def save_spectrogram_tisv(dict1):
    """ Full preprocess of text independent utterance. The log-mel-spectrogram is saved as numpy file.
        Each partial utterance is splitted by voice detection using DB
        and the first and the last 180 frames from each partial utterance are saved. 
        Need : utterance data set (VTCK)
    """
    print("start text independent utterance feature extraction")
    os.makedirs(dict1['train_path'], exist_ok=True)   # make folder to save train file
    #os.makedirs(dict1['train_path'], exist_ok=True)    # make folder to save test file
    utter_min_len = (dict1['tisv_frame'] * dict1['hop'] + dict1['window']) * dict1['sr']   # lower bound of utterance length
    total_speaker_num = len(audio_path)
    train_speaker_num= (total_speaker_num//10)*9            # split total data 90% train and 10% test
    print("total speaker number : %d"%total_speaker_num)
    print("train : %d, test : %d"%(train_speaker_num, total_speaker_num-train_speaker_num))
    for i, folder in enumerate(audio_path):
        print("%dth speaker processing..."%i)
        utterances_spec = []
        for utter_name in os.listdir(folder):
            if utter_name[-4:] == '.wav':
                utter_path = os.path.join(folder, utter_name)         # path of each utterance
                utter, sr = librosa.core.load(utter_path, dict1['sr'])        # load utterance audio
                intervals = librosa.effects.split(utter, top_db=60)       # voice activity detection
                for interval in intervals:
                    if (interval[1]-interval[0]) > utter_min_len:           # If partial utterance is sufficient long,
                        utter_part = utter[interval[0]:interval[1]]         # save first and last 180 frames of spectrogram.
                        S = librosa.core.stft(y=utter_part, n_fft=dict1['nfft'],
                                              win_length=int(dict1['window'] * dict1['sr']), hop_length=int(dict1['hop'] * dict1['sr']))
                        S = np.abs(S) ** 2
                        mel_basis = librosa.filters.mel(sr=dict1['sr'], n_fft=dict1['nfft'], n_mels=dict1['nmels'])
                        S = np.log10(np.dot(mel_basis, S) + 1e-6)           # log mel spectrogram of utterances
                        utterances_spec.append(S[:, :dict1['tisv_frame']])    # first 180 frames of partial utterance
                        utterances_spec.append(S[:, -dict1['tisv_frame']:])   # last 180 frames of partial utterance

        utterances_spec = np.array(utterances_spec)
        print("2",utterances_spec.shape)
        if i<train_speaker_num:      # save spectrogram as numpy file
            np.save(os.path.join(dict1['train_path'], "speaker%d.npy"%i), utterances_spec)

if __name__ == "__main__":
    save_spectrogram_tisv(dict1)

start text independent utterance feature extraction
total speaker number : 462
train : 414, test : 48
0th speaker processing...
2 (20, 40, 180)
1th speaker processing...
2 (20, 40, 180)
2th speaker processing...
2 (20, 40, 180)
3th speaker processing...
2 (20, 40, 180)
4th speaker processing...
2 (20, 40, 180)
5th speaker processing...
2 (20, 40, 180)
6th speaker processing...
2 (18, 40, 180)
7th speaker processing...
2 (20, 40, 180)
8th speaker processing...
2 (18, 40, 180)
9th speaker processing...
2 (20, 40, 180)
10th speaker processing...
2 (18, 40, 180)
11th speaker processing...
2 (20, 40, 180)
12th speaker processing...
2 (20, 40, 180)
13th speaker processing...
2 (20, 40, 180)
14th speaker processing...
2 (20, 40, 180)
15th speaker processing...
2 (20, 40, 180)
16th speaker processing...
2 (14, 40, 180)
17th speaker processing...
2 (20, 40, 180)
18th speaker processing...
2 (16, 40, 180)
19th speaker processing...
2 (20, 40, 180)
20th speaker processing...
2 (20, 40, 180)
21th 

In [13]:
import time as time
from torch.utils.data import DataLoader
def train(model_path,dict1):
    device = torch.device(dict1['device'])
    device=dict1['device']
    if dict1['data_preprocessed']:
        train_dataset = SpeakerDatasetTIMITPreprocessed(dict1)
    else:
        train_dataset = SpeakerDatasetTIMIT(dict1)
    train_loader = DataLoader(train_dataset, batch_size=dict1['N'], shuffle=True, num_workers=dict1['num_workers'], drop_last=True) 
    print(train_loader)
    embedder_net = SpeechEmbedder(dict1).to(device)
    if dict1['restore']:
        embedder_net.load_state_dict(torch.load(model_path))
    ge2e_loss = GE2ELoss(device)
    #Both net and loss have trainable parameters
    optimizer = torch.optim.SGD([
                    {'params': embedder_net.parameters()},
                    {'params': ge2e_loss.parameters()}
                ], lr=dict1['lr'])
    
    os.makedirs(dict1['checkpoint_dir'], exist_ok=True)
    
    embedder_net.train()
    iteration = 0
    for e in range(100):
        total_loss = 0
        for batch_id, mel_db_batch in enumerate(train_loader): 
            mel_db_batch = mel_db_batch.to(device)
            
            mel_db_batch = torch.reshape(mel_db_batch, (dict1['N']*dict1['M'], mel_db_batch.size(2), mel_db_batch.size(3)))
            perm = random.sample(range(0, dict1['N']*dict1['M']),dict1['N']*dict1['M'])
            unperm = list(perm)
            for i,j in enumerate(perm):
                unperm[j] = i
            mel_db_batch = mel_db_batch[perm]
            #gradient accumulates
            optimizer.zero_grad()
            
            embeddings = embedder_net(mel_db_batch)
            embeddings = embeddings[unperm]
            embeddings = torch.reshape(embeddings, (dict1['N'],dict1['M'], embeddings.size(1)))
            
            #get loss, call backward, step optimizer
            loss = ge2e_loss(embeddings) #wants (Speaker, Utterances, embedding)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(embedder_net.parameters(), 3.0)
            torch.nn.utils.clip_grad_norm_(ge2e_loss.parameters(), 1.0)
            optimizer.step()
            
            total_loss = total_loss + loss
            iteration += 1
            if (batch_id + 1) % dict1['log_interval'] == 0:
                mesg = "{0}\tEpoch:{1}[{2}/{3}],Iteration:{4}\tLoss:{5:.4f}\tTLoss:{6:.4f}\t\n".format(time.ctime(), e+1,
                        batch_id+1, len(train_dataset)//dict1['N'], iteration,loss, total_loss / (batch_id + 1))
                print(mesg)
                if dict1['log_file'] is not None:
                    with open(dict1['log_file'],'a') as f:
                        f.write(mesg)
                    
        
        if dict1['checkpoint_dir'] is not None and (e + 1) % dict1['checkpoint_interval'] == 0:
            embedder_net.eval().cpu()
            ckpt_model_filename = "ckpt_epoch_" + str(e+1) + "_batch_id_" + str(batch_id+1) + ".pth"
            ckpt_model_path = os.path.join(dict1['checkpoint_dir'], ckpt_model_filename)
            torch.save(embedder_net.state_dict(), ckpt_model_path)
            embedder_net.to(device).train()

    #save model
    embedder_net.eval().cpu()
    save_model_filename = "final_epoch_" + str(e + 1) + "_batch_id_" + str(batch_id + 1) + ".model"
    save_model_path = os.path.join(dict1['checkpoint_dir'], save_model_filename)
    torch.save(embedder_net.state_dict(), save_model_path)
    
    print("\nDone, trained model saved at", save_model_path)

"""def test(model_path):
    
    if hp.data.data_preprocessed:
        test_dataset = SpeakerDatasetTIMITPreprocessed()
    else:
        test_dataset = SpeakerDatasetTIMIT()
    test_loader = DataLoader(test_dataset, batch_size=N, shuffle=True, num_workers=num_workers, drop_last=True)
    
    embedder_net = SpeechEmbedder()
    embedder_net.load_state_dict(torch.load(model_path))
    embedder_net.eval()
    
    avg_EER = 0
    for e in range(hp.test.epochs):
        batch_avg_EER = 0
        for batch_id, mel_db_batch in enumerate(test_loader):
            assert hp.test.M % 2 == 0
            enrollment_batch, verification_batch = torch.split(mel_db_batch, int(mel_db_batch.size(1)/2), dim=1)
            
            enrollment_batch = torch.reshape(enrollment_batch, (N*M//2, enrollment_batch.size(2), enrollment_batch.size(3)))
            verification_batch = torch.reshape(verification_batch, (N*M//2, verification_batch.size(2), verification_batch.size(3)))
            
            perm = random.sample(range(0,verification_batch.size(0)), verification_batch.size(0))
            unperm = list(perm)
            for i,j in enumerate(perm):
                unperm[j] = i
                
            verification_batch = verification_batch[perm]
            enrollment_embeddings = embedder_net(enrollment_batch)
            verification_embeddings = embedder_net(verification_batch)
            verification_embeddings = verification_embeddings[unperm]
            
            enrollment_embeddings = torch.reshape(enrollment_embeddings, (hp.test.N, hp.test.M//2, enrollment_embeddings.size(1)))
            verification_embeddings = torch.reshape(verification_embeddings, (hp.test.N, hp.test.M//2, verification_embeddings.size(1)))
            
            enrollment_centroids = get_centroids(enrollment_embeddings)
            
            sim_matrix = get_cossim(verification_embeddings, enrollment_centroids)
            
            # calculating EER
            diff = 1; EER=0; EER_thresh = 0; EER_FAR=0; EER_FRR=0
            
            for thres in [0.01*i+0.5 for i in range(50)]:
                sim_matrix_thresh = sim_matrix>thres
                
                FAR = (sum([sim_matrix_thresh[i].float().sum()-sim_matrix_thresh[i,:,i].float().sum() for i in range(int(N))])
                /(N-1.0)/(float(M/2))/N)
    
                FRR = (sum([M/2-sim_matrix_thresh[i,:,i].float().sum() for i in range(int(N))])
                /(float(M/2))/N)
                
                # Save threshold when FAR = FRR (=EER)
                if diff> abs(FAR-FRR):
                    diff = abs(FAR-FRR)
                    EER = (FAR+FRR)/2
                    EER_thresh = thres
                    EER_FAR = FAR
                    EER_FRR = FRR
            batch_avg_EER += EER
            print("\nEER : %0.2f (thres:%0.2f, FAR:%0.2f, FRR:%0.2f)"%(EER,EER_thresh,EER_FAR,EER_FRR))
        avg_EER += batch_avg_EER/(batch_id+1)
    avg_EER = avg_EER / hp.test.epochs
    print("\n EER across {0} epochs: {1:.4f}".format(hp.test.epochs, avg_EER))"""
        
if __name__=="__main__":
    if 1==1:
        train(dict1['model_path'],dict1)
    else:
        test(hp.model.model_path)
#here we are only training 

Wed Oct  2 08:47:54 2019	Epoch:1[30/103],Iteration:30	Loss:33.0512	TLoss:32.5863	

Wed Oct  2 08:48:05 2019	Epoch:1[60/103],Iteration:60	Loss:30.0816	TLoss:32.1748	

Wed Oct  2 08:48:17 2019	Epoch:1[90/103],Iteration:90	Loss:32.7073	TLoss:31.0912	

Wed Oct  2 08:48:34 2019	Epoch:2[30/103],Iteration:133	Loss:24.4485	TLoss:28.5128	

Wed Oct  2 08:48:45 2019	Epoch:2[60/103],Iteration:163	Loss:24.4520	TLoss:28.6056	

Wed Oct  2 08:48:57 2019	Epoch:2[90/103],Iteration:193	Loss:32.1484	TLoss:27.9410	

Wed Oct  2 08:49:14 2019	Epoch:3[30/103],Iteration:236	Loss:25.2105	TLoss:26.4450	

Wed Oct  2 08:49:25 2019	Epoch:3[60/103],Iteration:266	Loss:21.3958	TLoss:26.1612	

Wed Oct  2 08:49:37 2019	Epoch:3[90/103],Iteration:296	Loss:23.1856	TLoss:25.6706	

Wed Oct  2 08:49:54 2019	Epoch:4[30/103],Iteration:339	Loss:21.9701	TLoss:25.3947	

Wed Oct  2 08:50:06 2019	Epoch:4[60/103],Iteration:369	Loss:33.0846	TLoss:25.1355	

Wed Oct  2 08:50:17 2019	Epoch:4[90/103],Iteration:399	Loss:31.5411	TLoss:24.84

In [0]:
_DEFAULT_OBSERVATION_DIM = 256

import argparse
def str2bool(value):
  """A function to convert string to bool value."""
  if value.lower() in {'yes', 'true', 't', 'y', '1'}:
    return True
  if value.lower() in {'no', 'false', 'f', 'n', '0'}:
    return False
# model configurations
class arguments:
    def __init__(self):
        self.observation_dim=_DEFAULT_OBSERVATION_DIM
        self.rnn_hidden_size=512
        self.rnn_depth=1
        self.rnn_dropout=0.2
        self.transition_bias=None
        self.crp_alpha=1.0
        self.sigma2=None
        self.verbosity=2
        self.enable_cuda=True
        self.optimizer='adam'
        self.learning_rate=1e-5
        self.train_iteration=20000
        self.batch_size=20
        self.num_permutations=10
        self.sigma_alpha=1
        self.sigma_beta=1
        self.regularization_weight=1e-5
        self.grad_max_norm=5
        self.enforce_cluster_id_uniqueness=True
        self.beam_size=10
        self.look_ahead=1
        self.test_iteration=2

In [0]:
import torch


def weighted_mse_loss(input_tensor, target_tensor, weight=1):
  """Compute weighted MSE loss.
  Note that we are doing weighted loss that only sum up over non-zero entries.
  Args:
    input_tensor: input tensor
    target_tensor: target tensor
    weight: weight tensor, in this case 1/sigma^2
  Returns:
    the weighted MSE loss
  """
  observation_dim = input_tensor.size()[-1]
  streched_tensor = ((input_tensor - target_tensor) ** 2).view(
      -1, observation_dim)
  entry_num = float(streched_tensor.size()[0])
  non_zero_entry_num = torch.sum(streched_tensor[:, 0] != 0).float()
  weighted_tensor = torch.mm(
      ((input_tensor - target_tensor)**2).view(-1, observation_dim),
      (torch.diag(weight.float().view(-1))))
  return torch.mean(
      weighted_tensor) * weight.nelement() * entry_num / non_zero_entry_num


def sigma2_prior_loss(num_non_zero, sigma_alpha, sigma_beta, sigma2):
    """Compute sigma2 prior loss.
  Args:
    num_non_zero: since rnn_truth is a collection of different length sequences
        padded with zeros to fit them into a tensor, we count the sum of
        'real lengths' of all sequences
    sigma_alpha: inverse gamma shape
    sigma_beta: inverse gamma scale
    sigma2: sigma squared
  Returns:
    the sigma2 prior loss
    """
    return ((2 * sigma_alpha + num_non_zero + 2) /
          (2 * num_non_zero) * torch.log(sigma2)).sum() + (
              sigma_beta / (sigma2 * num_non_zero)).sum()


def regularization_loss(params, weight):
  """Compute regularization loss.
  Args:
    params: iterable of all parameters
    weight: weight for the regularization term
  Returns:
    the regularization loss
  """
  l2_reg = 0
  for param in params:
    l2_reg += torch.norm(param)
  return weight * l2_reg

In [0]:
from scipy import optimize
import numpy as np


def get_list_inverse_index(unique_ids):
  """Get value to position index from a list of unique ids.
  Args:
    unique_ids: A list of unique integers of strings.
  Returns:
    result: a dict from value to position
  Raises:
    TypeError: If unique_ids is not a list.
  """
  if not isinstance(unique_ids, list):
    raise TypeError('unique_ids must be a list')
  result = dict()
  for i, unique_id in enumerate(unique_ids):
    result[unique_id] = i
  return result

def compute_sequence_match_accuracy(sequence1, sequence2):
  """Compute the accuracy between two sequences by finding optimal matching.
  Args:
    sequence1: A list of integers or strings.
    sequence2: A list of integers or strings.
  Returns:
    accuracy: sequence matching accuracy as a number in [0.0, 1.0]
  Raises:
    TypeError: If sequence1 or sequence2 is not list.
    ValueError: If sequence1 and sequence2 are not same size.
  """
  sequence1=list(sequence1)
  if not isinstance(sequence1, list) or not isinstance(sequence2, list):
    raise TypeError('sequence1 and sequence2 must be lists')
  if not sequence1 or len(sequence1) != len(sequence2):
    raise ValueError(
        'sequence1 and sequence2 must have the same non-zero length')
  # get unique ids from sequences
  unique_ids1 = sorted(set(sequence1))
  unique_ids2 = sorted(set(sequence2))
  inverse_index1 = get_list_inverse_index(unique_ids1)
  inverse_index2 = get_list_inverse_index(unique_ids2)
  print(sequence1)
  print(sequence2)
  print(inverse_index1)
  print(inverse_index2)
  # get the count matrix
  count_matrix = np.zeros((len(unique_ids1), len(unique_ids2)))
  for item1, item2 in zip(sequence1, sequence2):
    index1 = inverse_index1[item1]
    index2 = inverse_index2[item2]
    count_matrix[index1, index2] += 1.0
  row_index, col_index = optimize.linear_sum_assignment(-count_matrix)
  optimal_match_count = count_matrix[row_index, col_index].sum()
  accuracy = optimal_match_count / len(sequence1)
  return accuracy

In [0]:
import random
import string

import numpy as np
import torch
from torch import autograd


class Logger:
  """A class for printing logging information to screen."""

  def __init__(self, verbosity):
    self._verbosity = verbosity

  def print(self, level, message):
    """Print a message if level is not higher than verbosity.
    Args:
      level: the level of this message, smaller value means more important
      message: the message to be printed
    """
    if level <= self._verbosity:
      print(message)


def generate_random_string(length=6):
  """Generate a random string of upper case letters and digits.
  Args:
    length: length of the generated string
  Returns:
    the generated string
  """
  return ''.join([
      random.choice(string.ascii_uppercase + string.digits)
      for _ in range(length)])


def enforce_cluster_id_uniqueness(cluster_ids):
  """Enforce uniqueness of cluster id across sequences.
  Args:
    cluster_ids: a list of 1-dim list/numpy.ndarray of strings
  Returns:
    a new list with same length of cluster_ids
  Raises:
    TypeError: if cluster_ids or its element has wrong type
  """
  if not isinstance(cluster_ids, list):
    raise TypeError('cluster_ids must be a list')
  new_cluster_ids = []
  for cluster_id in cluster_ids:
    sequence_id = generate_random_string()
    if isinstance(cluster_id, np.ndarray):
      cluster_id = cluster_id.tolist()
    if not isinstance(cluster_id, list):
      raise TypeError('Elements of cluster_ids must be list or numpy.ndarray')
    new_cluster_id = ['_'.join([sequence_id, s]) for s in cluster_id]
    new_cluster_ids.append(new_cluster_id)
  return new_cluster_ids


def concatenate_training_data(train_sequences, train_cluster_ids,
                              enforce_uniqueness=True, shuffle=True):
  """Concatenate training data.
  Args:
    train_sequences: a list of 2-dim numpy arrays to be concatenated
    train_cluster_ids: a list of 1-dim list/numpy.ndarray of strings
    enforce_uniqueness: a boolean indicated whether we should enfore uniqueness
      to train_cluster_ids
    shuffle: whether to randomly shuffle input order
  Returns:
    concatenated_train_sequence: a 2-dim numpy array
    concatenated_train_cluster_id: a list of strings
  Raises:
    TypeError: if input has wrong type
    ValueError: if sizes/dimensions of input or their elements are incorrect
  """
  # check input
  if not isinstance(train_sequences, list) or not isinstance(
      train_cluster_ids, list):
    raise TypeError('train_sequences and train_cluster_ids must be lists')
  if len(train_sequences) != len(train_cluster_ids):
    raise ValueError(
        'train_sequences and train_cluster_ids must have same size')
  train_cluster_ids = [
      x.tolist() if isinstance(x, np.ndarray) else x
      for x in train_cluster_ids]
  global_observation_dim = None
  for i, (train_sequence, train_cluster_id) in enumerate(
      zip(train_sequences, train_cluster_ids)):
    train_length, observation_dim = train_sequence.shape
    if i == 0:
      global_observation_dim = observation_dim
    elif global_observation_dim != observation_dim:
      raise ValueError(
          'train_sequences must have consistent observation dimension')
    if not isinstance(train_cluster_id, list):
      raise TypeError(
          'Elements of train_cluster_ids must be list or numpy.ndarray')
    if len(train_cluster_id) != train_length:
      raise ValueError(
          'Each train_sequence and its train_cluster_id must have same length')

  # enforce uniqueness
  if enforce_uniqueness:
    train_cluster_ids = enforce_cluster_id_uniqueness(train_cluster_ids)

  # random shuffle
  if shuffle:
    zipped_input = list(zip(train_sequences, train_cluster_ids))
    random.shuffle(zipped_input)
    train_sequences, train_cluster_ids = zip(*zipped_input)

  # concatenate
  concatenated_train_sequence = np.concatenate(train_sequences, axis=0)
  concatenated_train_cluster_id = [x for train_cluster_id in train_cluster_ids
                                   for x in train_cluster_id]
  return concatenated_train_sequence, concatenated_train_cluster_id


def sample_permuted_segments(index_sequence, number_samples):
  """Sample sequences with permuted blocks.
  Args:
    index_sequence: (integer array, size: L)
      - subsequence index
      For example, index_sequence = [1,2,6,10,11,12].
    number_samples: (integer)
      - number of subsampled block-preserving permuted sequences.
      For example, number_samples = 5
  Returns:
    sampled_index_sequences: (a list of numpy arrays) - a list of subsampled
      block-preserving permuted sequences. For example,
    ```
    sampled_index_sequences =
    [[10,11,12,1,2,6],
     [6,1,2,10,11,12],
     [1,2,10,11,12,6],
     [6,1,2,10,11,12],
     [1,2,6,10,11,12]]
    ```
      The length of "sampled_index_sequences" is "number_samples".
  """
  segments = []
  if len(index_sequence) == 1:
    segments.append(index_sequence)
  else:
    prev = 0
    for i in range(len(index_sequence) - 1):
      if index_sequence[i + 1] != index_sequence[i] + 1:
        segments.append(index_sequence[prev:(i + 1)])
        prev = i + 1
      if i + 1 == len(index_sequence) - 1:
        segments.append(index_sequence[prev:])
  # sample permutations
  sampled_index_sequences = []
  for _ in range(number_samples):
    segments_array = []
    permutation = np.random.permutation(len(segments))
    for permutation_item in permutation:
      segments_array.append(segments[permutation_item])
    sampled_index_sequences.append(np.concatenate(segments_array))
  return sampled_index_sequences


def resize_sequence(sequence, cluster_id, num_permutations=None):
  """Resize sequences for packing and batching.
  Args:
    sequence: (real numpy matrix, size: seq_len*obs_size) - observed sequence
    cluster_id: (numpy vector, size: seq_len) - cluster indicator sequence
    num_permutations: int - Number of permutations per utterance sampled.
  Returns:
    sub_sequences: A list of numpy array, with obsevation vector from the same
      cluster in the same list.
    seq_lengths: The length of each cluster (+1).
    bias: Flipping coin head probability.
    bias_denominator: The denominator of the bias, used for multiple calls to
      fit().
  """
  # merge sub-sequences that belong to a single cluster to a single sequence
  unique_id = np.unique(cluster_id)
  sub_sequences = []
  seq_lengths = []
  if num_permutations and num_permutations > 1:
    for i in unique_id:
      idx_set = np.where(cluster_id == i)[0]
      sampled_idx_sets = sample_permuted_segments(idx_set, num_permutations)
      for j in range(num_permutations):
        sub_sequences.append(sequence[sampled_idx_sets[j], :])
        seq_lengths.append(len(idx_set) + 1)
  else:
    for i in unique_id:
      idx_set = np.where(cluster_id == i)
      sub_sequences.append(sequence[idx_set, :][0])
      seq_lengths.append(len(idx_set[0]) + 1)

  # compute bias
  transit_num = 0
  for entry in range(len(cluster_id) - 1):
    transit_num += (cluster_id[entry] != cluster_id[entry + 1])
  bias_denominator = len(cluster_id)
  bias = (transit_num + 1) / bias_denominator
  return sub_sequences, seq_lengths, bias, bias_denominator


def pack_sequence(
    sub_sequences, seq_lengths, batch_size, observation_dim, device):
  """Pack sequences for training.
  Args:
    sub_sequences: A list of numpy array, with obsevation vector from the same
      cluster in the same list.
    seq_lengths: The length of each cluster (+1).
    batch_size: int or None - Run batch learning if batch_size is None. Else,
      run online learning with specified batch size.
    observation_dim: int - dimension for observation vectors
    device: str - Your device. E.g., `cuda:0` or `cpu`.
  Returns:
    packed_rnn_input: (PackedSequence object) packed rnn input
    rnn_truth: ground truth
  """
  num_clusters = len(seq_lengths)
  sorted_seq_lengths = np.sort(seq_lengths)[::-1]
  permute_index = np.argsort(seq_lengths)[::-1]

  if batch_size is None:
    rnn_input = np.zeros((sorted_seq_lengths[0],
                          num_clusters,
                          observation_dim))
    for i in range(num_clusters):
      rnn_input[1:sorted_seq_lengths[i], i,
                :] = sub_sequences[permute_index[i]]
    rnn_input = autograd.Variable(
        torch.from_numpy(rnn_input).float()).to(device)
    packed_rnn_input = torch.nn.utils.rnn.pack_padded_sequence(
        rnn_input, sorted_seq_lengths, batch_first=False)
  else:
    mini_batch = np.sort(np.random.choice(num_clusters, batch_size))
    rnn_input = np.zeros((sorted_seq_lengths[mini_batch[0]],
                          batch_size,
                          observation_dim))
    for i in range(batch_size):
      rnn_input[1:sorted_seq_lengths[mini_batch[i]],
                i, :] = sub_sequences[permute_index[mini_batch[i]]]
    rnn_input = autograd.Variable(
        torch.from_numpy(rnn_input).float()).to(device)
    packed_rnn_input = torch.nn.utils.rnn.pack_padded_sequence(
        rnn_input, sorted_seq_lengths[mini_batch], batch_first=False)
  # ground truth is the shifted input
  rnn_truth = rnn_input[1:, :, :]
  return packed_rnn_input, rnn_truth


def output_result(model_args, training_args, test_record):
  """Produce a string to summarize the experiment."""
  accuracy_array, _ = zip(*test_record)
  total_accuracy = np.mean(accuracy_array)
  output_string = """
Config:
  sigma_alpha: {}
  sigma_beta: {}
  crp_alpha: {}
  learning rate: {}
  regularization: {}
  batch size: {}
Performance:
  averaged accuracy: {:.6f}
  accuracy numbers for all testing sequences:
  """.strip().format(
      training_args.sigma_alpha,
      training_args.sigma_beta,
      model_args.crp_alpha,
      training_args.learning_rate,
      training_args.regularization_weight,
      training_args.batch_size,
      total_accuracy)
  for accuracy in accuracy_array:
    output_string += '\n    {:.6f}'.format(accuracy)
  output_string += '\n' + '=' * 80 + '\n'
  filename = 'layer_{}_{}_{:.1f}_result.txt'.format(
      model_args.rnn_hidden_size,
      model_args.rnn_depth, model_args.rnn_dropout)
  with open(filename, 'a') as file_object:
    file_object.write(output_string)
  return output_string

In [0]:
_INITIAL_SIGMA2_VALUE = 0.1

from torch import nn,optim
class CoreRNN(nn.Module):
  """The core Recurent Neural Network used by UIS-RNN."""

  def __init__(self, input_dim, hidden_size, depth, observation_dim, dropout=0):
    super(CoreRNN, self).__init__()
    self.hidden_size = hidden_size
    if depth >= 2:
      self.gru = nn.GRU(input_dim, hidden_size, depth, dropout=dropout)
    else:
      self.gru = nn.GRU(input_dim, hidden_size, depth)
    self.linear_mean1 = nn.Linear(hidden_size, hidden_size)
    self.linear_mean2 = nn.Linear(hidden_size, observation_dim)

  def forward(self, input_seq, hidden=None):
    output_seq, hidden = self.gru(input_seq, hidden)
    if isinstance(output_seq, torch.nn.utils.rnn.PackedSequence):
      output_seq, _ = torch.nn.utils.rnn.pad_packed_sequence(
          output_seq, batch_first=False)
    mean = self.linear_mean2(F.relu(self.linear_mean1(output_seq)))
    return mean, hidden


class BeamState:
  """Structure that contains necessary states for beam search."""

  def __init__(self, source=None):
    if not source:
      self.mean_set = []
      self.hidden_set = []
      self.neg_likelihood = 0
      self.trace = []
      self.block_counts = []
    else:
      self.mean_set = source.mean_set.copy()
      self.hidden_set = source.hidden_set.copy()
      self.trace = source.trace.copy()
      self.block_counts = source.block_counts.copy()
      self.neg_likelihood = source.neg_likelihood

  def append(self, mean, hidden, cluster):
    """Append new item to the BeamState."""
    self.mean_set.append(mean.clone())
    self.hidden_set.append(hidden.clone())
    self.block_counts.append(1)
    self.trace.append(cluster)


class UISRNN:
  """Unbounded Interleaved-State Recurrent Neural Networks."""
  
  def __init__(self,args):
    """Construct the UISRNN object.
    Args:
      args: Model configurations. See `arguments.py` for details.
    """
    self.observation_dim = args.observation_dim
    self.device = torch.device(
        'cuda:0' if (torch.cuda.is_available() and args.enable_cuda) else 'cpu')
    self.rnn_model = CoreRNN(self.observation_dim, args.rnn_hidden_size,
                             args.rnn_depth, self.observation_dim,
                             args.rnn_dropout).to(self.device)
    self.rnn_init_hidden = nn.Parameter(
        torch.zeros(args.rnn_depth, 1, args.rnn_hidden_size).to(self.device))
    # booleans indicating which variables are trainable
    self.estimate_sigma2 = (args.sigma2 is None)
    self.estimate_transition_bias = (args.transition_bias is None)
    # initial values of variables
    sigma2 = _INITIAL_SIGMA2_VALUE if self.estimate_sigma2 else args.sigma2
    self.sigma2 = nn.Parameter(
        sigma2 * torch.ones(self.observation_dim).to(self.device))
    self.transition_bias = args.transition_bias
    self.transition_bias_denominator = 0.0
    self.crp_alpha = args.crp_alpha
    self.logger =Logger(args.verbosity)

  def _get_optimizer(self, optimizer, learning_rate):
    """Get optimizer for UISRNN.
    Args:
      optimizer: string - name of the optimizer.
      learning_rate: - learning rate for the entire model.
        We do not customize learning rate for separate parts.
    Returns:
      a pytorch "optim" object
    """
    params = [
        {
            'params': self.rnn_model.parameters()
        },  # rnn parameters
        {
            'params': self.rnn_init_hidden
        }  # rnn initial hidden state
    ]
    if self.estimate_sigma2:  # train sigma2
      params.append({
          'params': self.sigma2
      })  # variance parameters
    assert optimizer == 'adam', 'Only adam optimizer is supported.'
    return optim.Adam(params, lr=learning_rate)

  def save(self, filepath):
    """Save the model to a file.
    Args:
      filepath: the path of the file.
    """
    torch.save({
        'rnn_state_dict': self.rnn_model.state_dict(),
        'rnn_init_hidden': self.rnn_init_hidden.detach().cpu().numpy(),
        'transition_bias': self.transition_bias,
        'transition_bias_denominator': self.transition_bias_denominator,
        'crp_alpha': self.crp_alpha,
        'sigma2': self.sigma2.detach().cpu().numpy()}, filepath)

  def load(self, filepath):
    """Load the model from a file.
    Args:
      filepath: the path of the file.
    """
    var_dict = torch.load(filepath)
    self.rnn_model.load_state_dict(var_dict['rnn_state_dict'])
    self.rnn_init_hidden = nn.Parameter(
        torch.from_numpy(var_dict['rnn_init_hidden']).to(self.device))
    self.transition_bias = float(var_dict['transition_bias'])
    self.transition_bias_denominator = float(
        var_dict['transition_bias_denominator'])
    self.crp_alpha = float(var_dict['crp_alpha'])
    self.sigma2 = nn.Parameter(
        torch.from_numpy(var_dict['sigma2']).to(self.device))

    self.logger.print(
        3, 'Loaded model with transition_bias={}, crp_alpha={}, sigma2={}, '
        'rnn_init_hidden={}'.format(
            self.transition_bias, self.crp_alpha, var_dict['sigma2'],
            var_dict['rnn_init_hidden']))

  def fit_concatenated(self, train_sequence, train_cluster_id, args):
    """Fit UISRNN model to concatenated sequence and cluster_id.
    Args:
      train_sequence: the training observation sequence, which is a
        2-dim numpy array of real numbers, of size `N * D`.
        - `N`: summation of lengths of all utterances.
        - `D`: observation dimension.
        For example,
      ```
      train_sequence =
      [[1.2 3.0 -4.1 6.0]    --> an entry of speaker #0 from utterance 'iaaa'
       [0.8 -1.1 0.4 0.5]    --> an entry of speaker #1 from utterance 'iaaa'
       [-0.2 1.0 3.8 5.7]    --> an entry of speaker #0 from utterance 'iaaa'
       [3.8 -0.1 1.5 2.3]    --> an entry of speaker #0 from utterance 'ibbb'
       [1.2 1.4 3.6 -2.7]]   --> an entry of speaker #0 from utterance 'ibbb'
      ```
        Here `N=5`, `D=4`.
        We concatenate all training utterances into this single sequence.
      train_cluster_id: the speaker id sequence, which is 1-dim list or
        numpy array of strings, of size `N`.
        For example,
      ```
      train_cluster_id =
        ['iaaa_0', 'iaaa_1', 'iaaa_0', 'ibbb_0', 'ibbb_0']
      ```
        'iaaa_0' means the entry belongs to speaker #0 in utterance 'iaaa'.
        Note that the order of entries within an utterance are preserved,
        and all utterances are simply concatenated together.
      args: Training configurations. See `arguments.py` for details.
    Raises:
      TypeError: If train_sequence or train_cluster_id is of wrong type.
      ValueError: If train_sequence or train_cluster_id has wrong dimension.
    """
    # check type
    if (not isinstance(train_sequence, np.ndarray) or
        train_sequence.dtype != float):
      raise TypeError('train_sequence should be a numpy array of float type.')
    if isinstance(train_cluster_id, list):
      train_cluster_id = np.array(train_cluster_id)
    if (not isinstance(train_cluster_id, np.ndarray) or
        not train_cluster_id.dtype.name.startswith(('str', 'unicode'))):
      raise TypeError('train_cluster_id type be a numpy array of strings.')
    # check dimension
    if train_sequence.ndim != 2:
      raise ValueError('train_sequence must be 2-dim array.')
    if train_cluster_id.ndim != 1:
      raise ValueError('train_cluster_id must be 1-dim array.')
    # check length and size
    train_total_length, observation_dim = train_sequence.shape
    if observation_dim != self.observation_dim:
      raise ValueError('train_sequence does not match the dimension specified '
                       'by args.observation_dim.')
    if train_total_length != len(train_cluster_id):
      raise ValueError('train_sequence length is not equal to '
                       'train_cluster_id length.')

    self.rnn_model.train()
    optimizer = self._get_optimizer(optimizer=args.optimizer,
                                    learning_rate=args.learning_rate)

    (sub_sequences,
     seq_lengths,
     transition_bias,
     transition_bias_denominator) = resize_sequence(
         sequence=train_sequence,
         cluster_id=train_cluster_id,
         num_permutations=args.num_permutations)
    if self.estimate_transition_bias:
      if self.transition_bias is None:
        self.transition_bias = transition_bias
        self.transition_bias_denominator = transition_bias_denominator
        
      else:
        
        self.transition_bias = (
            self.transition_bias * self.transition_bias_denominator +
            transition_bias * transition_bias_denominator) / (
                self.transition_bias_denominator + transition_bias_denominator)
        self.transition_bias_denominator += transition_bias_denominator

    # For batch learning, pack the entire dataset.
    if args.batch_size is None:
      packed_train_sequence, rnn_truth = utils.pack_sequence(
          sub_sequences,
          seq_lengths,
          args.batch_size,
          self.observation_dim,
          self.device)
    train_loss = []
    for num_iter in range(args.train_iteration):
      optimizer.zero_grad()
      # For online learning, pack a subset in each iteration.
      if args.batch_size is not None:
        packed_train_sequence, rnn_truth = pack_sequence(
            sub_sequences,
            seq_lengths,
            args.batch_size,
            self.observation_dim,
            self.device)
      hidden = self.rnn_init_hidden.repeat(1, args.batch_size, 1)
      mean, _ = self.rnn_model(packed_train_sequence, hidden)
      # use mean to predict
      mean = torch.cumsum(mean, dim=0)
      mean_size = mean.size()
      mean = torch.mm(
          torch.diag(
              1.0 / torch.arange(1, mean_size[0] + 1).float().to(self.device)),
          mean.view(mean_size[0], -1))
      mean = mean.view(mean_size)

      # Likelihood part.
      loss1 = weighted_mse_loss(
          input_tensor=(rnn_truth != 0).float() * mean[:-1, :, :],
          target_tensor=rnn_truth,
          weight=1 / (2 * self.sigma2))

      # Sigma2 prior part.
      weight = (((rnn_truth != 0).float() * mean[:-1, :, :] - rnn_truth)
                ** 2).view(-1, observation_dim)
      num_non_zero = torch.sum((weight != 0).float(), dim=0).squeeze()
      loss2 = sigma2_prior_loss(
          num_non_zero, args.sigma_alpha, args.sigma_beta, self.sigma2)

      # Regularization part.
      loss3 = regularization_loss(
          self.rnn_model.parameters(), args.regularization_weight)

      loss = loss1 + loss2 + loss3
      loss.backward()
      nn.utils.clip_grad_norm_(self.rnn_model.parameters(), args.grad_max_norm)
      optimizer.step()
      # avoid numerical issues
      self.sigma2.data.clamp_(min=1e-6)

      if (np.remainder(num_iter, 10) == 0 or
          num_iter == args.train_iteration - 1):
        self.logger.print(
            2,
            'Iter: {:d}  \t'
            'Training Loss: {:.4f}    \n'
            '    Negative Log Likelihood: {:.4f}\t'
            'Sigma2 Prior: {:.4f}\t'
            'Regularization: {:.4f}'.format(
                num_iter,
                float(loss.data),
                float(loss1.data),
                float(loss2.data),
                float(loss3.data)))
      train_loss.append(float(loss1.data))  # only save the likelihood part
    self.logger.print(
        1, 'Done training with {} iterations'.format(args.train_iteration))

  def fit(self, train_sequences, train_cluster_ids, args):
    """Fit UISRNN model.
    Args:
      train_sequences: Either a list of training sequences, or a single
        concatenated training sequence:
        1. train_sequences is list, and each element is a 2-dim numpy array
           of real numbers, of size: `length * D`.
           The length varies among different sequences, but the D is the same.
           In speaker diarization, each sequence is the sequence of speaker
           embeddings of one utterance.
        2. train_sequences is a single concatenated sequence, which is a
           2-dim numpy array of real numbers. See `fit_concatenated()`
           for more details.
      train_cluster_ids: Ground truth labels for train_sequences:
        1. if train_sequences is a list, this must also be a list of the same
           size, each element being a 1-dim list or numpy array of strings.
        2. if train_sequences is a single concatenated sequence, this
           must also be the concatenated 1-dim list or numpy array of strings
      args: Training configurations. See `arguments.py` for details.
    Raises:
      TypeError: If train_sequences or train_cluster_ids is of wrong type.
    """
    if isinstance(train_sequences, np.ndarray):
      # train_sequences is already the concatenated sequence
      concatenated_train_sequence = train_sequences
      concatenated_train_cluster_id = train_cluster_ids
    elif isinstance(train_sequences, list):
      # train_sequences is a list of un-concatenated sequences,
      # then we concatenate them first
      (concatenated_train_sequence,
       concatenated_train_cluster_id) = utils.concatenate_training_data(
           train_sequences,
           train_cluster_ids,
           args.enforce_cluster_id_uniqueness,
           True)
    else:
      raise TypeError('train_sequences must be a list or numpy.ndarray')

    self.fit_concatenated(
        concatenated_train_sequence, concatenated_train_cluster_id, args)

  def _update_beam_state(self, beam_state, look_ahead_seq, cluster_seq):
    """Update a beam state given a look ahead sequence and known cluster
    assignments.
    Args:
      beam_state: A BeamState object.
      look_ahead_seq: Look ahead sequence, size: look_ahead*D.
        look_ahead: number of step to look ahead in the beam search.
        D: observation dimension
      cluster_seq: Cluster assignment sequence for look_ahead_seq.
    Returns:
      new_beam_state: An updated BeamState object.
    """

    loss = 0
    new_beam_state = BeamState(beam_state)
    for sub_idx, cluster in enumerate(cluster_seq):
      if cluster > len(new_beam_state.mean_set):  # invalid trace
        new_beam_state.neg_likelihood = float('inf')
        break
      elif cluster < len(new_beam_state.mean_set):  # existing cluster
        last_cluster = new_beam_state.trace[-1]
        loss = weighted_mse_loss(
            input_tensor=torch.squeeze(new_beam_state.mean_set[cluster]),
            target_tensor=look_ahead_seq[sub_idx, :],
            weight=1 / (2 * self.sigma2)).cpu().detach().numpy()
        if cluster == last_cluster:
          loss -= np.log(1 - self.transition_bias)
        else:
          loss -= np.log(self.transition_bias) + np.log(
              new_beam_state.block_counts[cluster]) - np.log(
                  sum(new_beam_state.block_counts) + self.crp_alpha)
        # update new mean and new hidden
        mean, hidden = self.rnn_model(
            look_ahead_seq[sub_idx, :].unsqueeze(0).unsqueeze(0),
            new_beam_state.hidden_set[cluster])
        new_beam_state.mean_set[cluster] = (new_beam_state.mean_set[cluster]*(
            (np.array(new_beam_state.trace) == cluster).sum() -
            1).astype(float) + mean.clone()) / (
                np.array(new_beam_state.trace) == cluster).sum().astype(
                    float)  # use mean to predict
        new_beam_state.hidden_set[cluster] = hidden.clone()
        if cluster != last_cluster:
          new_beam_state.block_counts[cluster] += 1
        new_beam_state.trace.append(cluster)
      else:  # new cluster
        init_input = autograd.Variable(
            torch.zeros(self.observation_dim)
        ).unsqueeze(0).unsqueeze(0).to(self.device)
        mean, hidden = self.rnn_model(init_input,
                                      self.rnn_init_hidden)
        loss = weighted_mse_loss(
            input_tensor=torch.squeeze(mean),
            target_tensor=look_ahead_seq[sub_idx, :],
            weight=1 / (2 * self.sigma2)).cpu().detach().numpy()
        loss -= np.log(self.transition_bias) + np.log(
            self.crp_alpha) - np.log(
                sum(new_beam_state.block_counts) + self.crp_alpha)
        # update new min and new hidden
        mean, hidden = self.rnn_model(
            look_ahead_seq[sub_idx, :].unsqueeze(0).unsqueeze(0),
            hidden)
        new_beam_state.append(mean, hidden, cluster)
      new_beam_state.neg_likelihood += loss
    return new_beam_state

  def _calculate_score(self, beam_state, look_ahead_seq):
    """Calculate negative log likelihoods for all possible state allocations
       of a look ahead sequence, according to the current beam state.
    Args:
      beam_state: A BeamState object.
      look_ahead_seq: Look ahead sequence, size: look_ahead*D.
        look_ahead: number of step to look ahead in the beam search.
        D: observation dimension
    Returns:
      beam_score_set: a set of scores for each possible state allocation.
    """

    look_ahead, _ = look_ahead_seq.shape
    beam_num_clusters = len(beam_state.mean_set)
    beam_score_set = float('inf') * np.ones(
        beam_num_clusters + 1 + np.arange(look_ahead))
    for cluster_seq, _ in np.ndenumerate(beam_score_set):
      updated_beam_state = self._update_beam_state(beam_state,
                                                   look_ahead_seq, cluster_seq)
      beam_score_set[cluster_seq] = updated_beam_state.neg_likelihood
    return beam_score_set

  def predict_single(self, test_sequence, args):
    """Predict labels for a single test sequence using UISRNN model.
    Args:
      test_sequence: the test observation sequence, which is 2-dim numpy array
        of real numbers, of size `N * D`.
        - `N`: length of one test utterance.
        - `D` : observation dimension.
        For example:
      ```
      test_sequence =
      [[2.2 -1.0 3.0 5.6]    --> 1st entry of utterance 'iccc'
       [0.5 1.8 -3.2 0.4]    --> 2nd entry of utterance 'iccc'
       [-2.2 5.0 1.8 3.7]    --> 3rd entry of utterance 'iccc'
       [-3.8 0.1 1.4 3.3]    --> 4th entry of utterance 'iccc'
       [0.1 2.7 3.5 -1.7]]   --> 5th entry of utterance 'iccc'
      ```
        Here `N=5`, `D=4`.
      args: Inference configurations. See `arguments.py` for details.
    Returns:
      predicted_cluster_id: predicted speaker id sequence, which is
        an array of integers, of size `N`.
        For example, `predicted_cluster_id = [0, 1, 0, 0, 1]`
    Raises:
      TypeError: If test_sequence is of wrong type.
      ValueError: If test_sequence has wrong dimension.
    """
    # check type
    if (not isinstance(test_sequence, np.ndarray) or
        test_sequence.dtype != float):
      raise TypeError('test_sequence should be a numpy array of float type.')
    # check dimension
    if test_sequence.ndim != 2:
      raise ValueError('test_sequence must be 2-dim array.')
    # check size
    test_sequence_length, observation_dim = test_sequence.shape
    if observation_dim != self.observation_dim:
      raise ValueError('test_sequence does not match the dimension specified '
                       'by args.observation_dim.')

    self.rnn_model.eval()
    test_sequence = np.tile(test_sequence, (args.test_iteration, 1))
    test_sequence = autograd.Variable(
        torch.from_numpy(test_sequence).float()).to(self.device)
    # bookkeeping for beam search
    beam_set = [BeamState()]
    for num_iter in np.arange(0, args.test_iteration * test_sequence_length,
                              args.look_ahead):
      max_clusters = max([len(beam_state.mean_set) for beam_state in beam_set])
      look_ahead_seq = test_sequence[num_iter:  num_iter + args.look_ahead, :]
      look_ahead_seq_length = look_ahead_seq.shape[0]
      score_set = float('inf') * np.ones(
          np.append(
              args.beam_size, max_clusters + 1 + np.arange(
                  look_ahead_seq_length)))
      for beam_rank, beam_state in enumerate(beam_set):
        beam_score_set = self._calculate_score(beam_state, look_ahead_seq)
        score_set[beam_rank, :] = np.pad(
            beam_score_set,
            np.tile([[0, max_clusters - len(beam_state.mean_set)]],
                    (look_ahead_seq_length, 1)), 'constant',
            constant_values=float('inf'))
      # find top scores
      score_ranked = np.sort(score_set, axis=None)
      score_ranked[score_ranked == float('inf')] = 0
      score_ranked = np.trim_zeros(score_ranked)
      idx_ranked = np.argsort(score_set, axis=None)
      updated_beam_set = []
      for new_beam_rank in range(
          np.min((len(score_ranked), args.beam_size))):
        total_idx = np.unravel_index(idx_ranked[new_beam_rank],
                                     score_set.shape)
        prev_beam_rank = total_idx[0]
        cluster_seq = total_idx[1:]
        updated_beam_state = self._update_beam_state(
            beam_set[prev_beam_rank], look_ahead_seq, cluster_seq)
        updated_beam_set.append(updated_beam_state)
      beam_set = updated_beam_set
    predicted_cluster_id = beam_set[0].trace[-test_sequence_length:]
    return predicted_cluster_id

  def predict(self, test_sequences, args):
    """Predict labels for a single or many test sequences using UISRNN model.
    Args:
      test_sequences: Either a list of test sequences, or a single test
        sequence. Each test sequence is a 2-dim numpy array
        of real numbers. See `predict_single()` for details.
      args: Inference configurations. See `arguments.py` for details.
    Returns:
      predicted_cluster_ids: Predicted labels for test_sequences.
        1. if test_sequences is a list, predicted_cluster_ids will be a list
           of the same size, where each element being a 1-dim list of strings.
        2. if test_sequences is a single sequence, predicted_cluster_ids will
           be a 1-dim list of strings
    Raises:
      TypeError: If test_sequences is of wrong type.
    """
    # check type
    if isinstance(test_sequences, np.ndarray):
      return self.predict_single(test_sequences, args)
    if isinstance(test_sequences, list):
      return [self.predict_single(test_sequence, args)
              for test_sequence in test_sequences]
    raise TypeError('test_sequences should be either a list or numpy array.')

In [22]:
import glob
def concat_segs(times, segs):
    #Concatenate continuous voiced segments
    concat_seg = []
    seg_concat = segs[0]
    for i in range(0, len(times)-1):
        if times[i][1] == times[i+1][0]:
            seg_concat = np.concatenate((seg_concat, segs[i+1]))
        else:
            concat_seg.append(seg_concat)
            seg_concat = segs[i+1]
    else:
        concat_seg.append(seg_concat)
    return concat_seg

def get_STFTs(segs,dict1):
    #Get 240ms STFT windows with 50% overlap
    sr = 16000
    STFT_frames = []
    for seg in segs:
        S = librosa.core.stft(y=seg, n_fft=dict1['nfft'],
                              win_length=int(dict1['window'] * sr), hop_length=int(dict1['hop'] * sr))
        S = np.abs(S)**2
        mel_basis = librosa.filters.mel(sr, n_fft=dict1['nfft'], n_mels=dict1['nmels'])
        S = np.log10(np.dot(mel_basis, S) + 1e-6)           # log mel spectrogram of utterances
        for j in range(0, S.shape[1], int(.12/dict1['hop'])):
            if j + 24 < S.shape[1]:
                STFT_frames.append(S[:,j:j+24])
            else:
                break
    return STFT_frames

def align_embeddings(embeddings):
    partitions = []
    start = 0
    end = 0
    j = 1
    for i, embedding in enumerate(embeddings):
        if (i*.12)+.24 < j*.401:
            end = end + 1
        else:
            partitions.append((start,end))
            start = end
            end = end + 1
            j += 1
    else:
        partitions.append((start,end))
    avg_embeddings = np.zeros((len(partitions),256))
    for i, partition in enumerate(partitions):
        avg_embeddings[i] = np.average(embeddings[partition[0]:partition[1]],axis=0) 
    return avg_embeddings
#dataset path
audio_path = glob.glob(os.path.dirname(dict1['unprocessed_data']))  

total_speaker_num = len(audio_path)
train_speaker_num= (total_speaker_num//10)*9            # split total data 90% train and 10% test

embedder_net = SpeechEmbedder(dict1)
embedder_net.load_state_dict(torch.load("speech_id_checkpoint/final_epoch_100_batch_id_103.model"))
embedder_net.eval()

train_sequence = []
train_cluster_id = []
label = 0
count = 0
train_saved = False
for i, folder in enumerate(audio_path):
    for file in os.listdir(folder):
        if file[-4:] == '.wav':
            times, segs = VAD_chunk(2, folder+'/'+file)
            if segs == []:
                print('No voice activity detected')
                continue
            concat_seg = concat_segs(times, segs)
            STFT_frames = get_STFTs(concat_seg,dict1)
            STFT_frames = np.stack(STFT_frames, axis=2)
            STFT_frames = torch.tensor(np.transpose(STFT_frames, axes=(2,1,0)))
            embeddings = embedder_net(STFT_frames)
            aligned_embeddings = align_embeddings(embeddings.detach().numpy())
            train_sequence.append(aligned_embeddings)
            for embedding in aligned_embeddings:
                train_cluster_id.append(str(label))
            count = count + 1
            if count % 100 == 0:
                print('Processed {0}/{1} files'.format(count, len(audio_path)))
    label = label + 1
    
    if not train_saved and i > train_speaker_num:
        train_sequence = np.concatenate(train_sequence,axis=0)
        train_cluster_id = np.asarray(train_cluster_id)
        np.save('train_sequence',train_sequence)
        np.save('train_cluster_id',train_cluster_id)
        train_saved = True
        train_sequence = []
        train_cluster_id = []
        
train_sequence = np.concatenate(train_sequence,axis=0)
train_cluster_id = np.asarray(train_cluster_id)
np.save('test_sequence',train_sequence)
np.save('test_cluster_id',train_cluster_id)

Processed 100/462 files
Processed 200/462 files
Processed 300/462 files
Processed 400/462 files
Processed 500/462 files
Processed 600/462 files
Processed 700/462 files
Processed 800/462 files
Processed 900/462 files
Processed 1000/462 files
Processed 1100/462 files
Processed 1200/462 files
Processed 1300/462 files
Processed 1400/462 files
Processed 1500/462 files
Processed 1600/462 files
Processed 1700/462 files
Processed 1800/462 files
Processed 1900/462 files
Processed 2000/462 files
Processed 2100/462 files
Processed 2200/462 files
Processed 2300/462 files
Processed 2400/462 files
Processed 2500/462 files
Processed 2600/462 files
Processed 2700/462 files
Processed 2800/462 files
Processed 2900/462 files
Processed 3000/462 files
Processed 3100/462 files
Processed 3200/462 files
Processed 3300/462 files
Processed 3400/462 files
Processed 3500/462 files
Processed 3600/462 files
Processed 3700/462 files
Processed 3800/462 files
Processed 3900/462 files
Processed 4000/462 files
Processed

In [0]:
import numpy as np

SAVE_MODEL_NAME = 'saved_model_timit.uisrnn'
#model training on timit dataset

def diarization_experiment(args):
  """Experiment pipeline.
  Load data --> train model --> test model --> output result
  Args:
    model_args: model configurations
    training_args: training configurations
    inference_args: inference configurations
  """

  predicted_cluster_ids = []
  test_record = []

  train_sequence = np.load('train_sequence.npy')
  test_sequence= np.load('test_sequence.npy')
  train_cluster_id= np.load('train_cluster_id.npy')
  test_cluster_id= np.load('test_cluster_id.npy')
  test_cluster_id=test_cluster_id.reshape(-1,1)
  model = UISRNN(args)
  # training
  model.fit(train_sequence, train_cluster_id, args)
  model.save(SAVE_MODEL_NAME)
  # we can also skip training by calling：
  #model.load(SAVED_MODEL_NAME)

  # testing
  for (test_sequence, test_cluster_id) in zip(test_sequence, test_cluster_id):
    predicted_cluster_id = model.predict(test_sequence.reshape(1,-1), args)
    predicted_cluster_ids.append(predicted_cluster_id)
    print(type(predicted_cluster_id))
    accuracy = compute_sequence_match_accuracy(
        test_cluster_id, predicted_cluster_id)
    test_record.append((accuracy, len(test_cluster_id)))
    print('Ground truth labels:')
    print(test_cluster_id)
    print('Predicted labels:')
    print(predicted_cluster_id)
    print('-' * 80)
  output_string = output_result(args,args,test_record)

  print('Finished diarization experiment')
  print(output_string)
args=arguments()
diarization_experiment(args)

In [11]:
#test toy set
import numpy as np
import torch.nn.functional as F
import pickle
# training model on toy dataset
SAVED_MODEL_NAME = 'saved_model_toy.uisrnn'
test=np.load('drive/My Drive/toy_testing_data.npz',allow_pickle=True)
train=np.load('drive/My Drive/toy_training_data.npz',allow_pickle=True)
def diarization_experiment(args,train,test):
  """Experiment pipeline.
  Load data --> train model --> test model --> output result
  Args:
    model_args: model configurations
    training_args: training configurations
    inference_args: inference configurations
  """

  predicted_cluster_ids = []
  test_record = []

  train_sequence = train[train.files[0]]
  test_sequence= test[test.files[0]]
  train_cluster_id= train[train.files[1]]
  test_cluster_id= test[test.files[1]]
  #test_cluster_id=test_cluster_id.reshape(-1,1)
  model = UISRNN(args)
  # training
  model.fit(train_sequence, train_cluster_id, args)
  model.save(SAVED_MODEL_NAME)
  # we can also skip training by calling：
  #model.load(SAVED_MODEL_NAME)
  # testing
  for (test_sequence, test_cluster_id) in zip(test_sequence, test_cluster_id):
    predicted_cluster_id = model.predict(test_sequence, args)
    predicted_cluster_ids.append(predicted_cluster_id)
    print(type(predicted_cluster_id))
    accuracy = compute_sequence_match_accuracy(
        test_cluster_id, predicted_cluster_id)
    test_record.append((accuracy, len(test_cluster_id)))
    print('Ground truth labels:')
    print(test_cluster_id)
    print('Predicted labels:')
    print(predicted_cluster_id)
    print('-' * 80)
  output_string = output_result(args,args,test_record)

  print('Finished diarization experiment')
  print(output_string)
args=arguments()
diarization_experiment(args,train,test)

Iter: 0  	Training Loss: -285.9686    
    Negative Log Likelihood: 6.1656	Sigma2 Prior: -292.1349	Regularization: 0.0006
Iter: 10  	Training Loss: -286.9801    
    Negative Log Likelihood: 6.0607	Sigma2 Prior: -293.0415	Regularization: 0.0006
Iter: 20  	Training Loss: -287.5458    
    Negative Log Likelihood: 5.8720	Sigma2 Prior: -293.4185	Regularization: 0.0006
Iter: 30  	Training Loss: -287.0736    
    Negative Log Likelihood: 5.9862	Sigma2 Prior: -293.0605	Regularization: 0.0006
Iter: 40  	Training Loss: -287.5175    
    Negative Log Likelihood: 5.8229	Sigma2 Prior: -293.3410	Regularization: 0.0006
Iter: 50  	Training Loss: -287.4049    
    Negative Log Likelihood: 5.7267	Sigma2 Prior: -293.1323	Regularization: 0.0006
Iter: 60  	Training Loss: -287.3828    
    Negative Log Likelihood: 5.7282	Sigma2 Prior: -293.1116	Regularization: 0.0006
Iter: 70  	Training Loss: -287.9607    
    Negative Log Likelihood: 5.7012	Sigma2 Prior: -293.6625	Regularization: 0.0006
Iter: 80  	Traini

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#save model to drive
# 1. Authenticate and create the PyDrive client.
def save_files_on_drive(filename,ismodel):
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    if ismodel==True:
        model = UISRNN(args)
        model.load(filename)
        # create on Colab directory
        model.save(filename)    
        files = drive.CreateFile({'title' : filename})
        files.SetContentFile(filename)
        files.Upload()
        # download to google drive
        drive.CreateFile({'id': files.get('id')})
    if ismodel==False:
        d=np.load(filename)
        files = drive.CreateFile({'title' : filename})
        files.SetContentFile(d)
        files.Upload()
        # download to google drive
        drive.CreateFile({'id': files.get('id')})
    return 
save_files_on_drive("saved_model_toy.uisrnn",True)
save_files_on_drive("saved_model_timit.uisrnn",True)
save_files_on_drive("train_sequence.npy",False)
save_files_on_drive("test_sequence.npy",False)
save_files_on_drive("train_cluster_id.npy",False)
save_files_on_drive("test_cluster_id.npy",False)

In [0]:
import numpy as np

SAVED_MODEL_NAME = '/content/drive/My Drive/saved_model_toy.uisrnn'
#model trained on toy data set produced same results as the model trained on timit dataset

def diarization_experiment(args):
  """Experiment pipeline.
  Load data --> train model --> test model --> output result
  Args:
    model_args: model configurations
    training_args: training configurations
    inference_args: inference configurations
  """

  predicted_cluster_ids = []
  test_record = []

  train_sequence = np.load('/content/drive/My Drive/train_sequence.npy')
  test_sequence= np.load('/content/drive/My Drive/test_sequence.npy')
  train_cluster_id= np.load('/content/drive/My Drive/train_cluster_id.npy')
  test_cluster_id= np.load('/content/drive/My Drive/test_cluster_id.npy')
  test_cluster_id=test_cluster_id.reshape(-1,1)
  model = UISRNN(args)
  # training
  #model.fit(train_sequence, train_cluster_id, args)
  #model.save(SAVED_MODEL_NAME)
  # we can also skip training by calling：
  model.load(SAVED_MODEL_NAME)

  # testing
  for (test_sequence, test_cluster_id) in zip(test_sequence, test_cluster_id):
    predicted_cluster_id = model.predict(test_sequence.reshape(1,-1), args)
    predicted_cluster_ids.append(predicted_cluster_id)
    print(type(predicted_cluster_id))
    accuracy = compute_sequence_match_accuracy(
        test_cluster_id, predicted_cluster_id)
    test_record.append((accuracy, len(test_cluster_id)))
    print('Ground truth labels:')
    print(test_cluster_id)
    print('Predicted labels:')
    print(predicted_cluster_id)
    print('-' * 80)
  output_string = output_result(args,args,test_record)

  print('Finished diarization experiment')
  print(output_string)
args=arguments()
diarization_experiment(args)

In [0]:
#test toy set
import numpy as np
import torch.nn.functional as F
import pickle
# use trained model on timit to see performence on toy dataset
SAVED_MODEL_NAME = '/content/drive/My Drive/saved_model_timit.uisrnn'
test=np.load('drive/My Drive/toy_testing_data.npz',allow_pickle=True)
train=np.load('drive/My Drive/toy_training_data.npz',allow_pickle=True)
def diarization_experiment(args,train,test):
  """Experiment pipeline.
  Load data --> train model --> test model --> output result
  Args:
    model_args: model configurations
    training_args: training configurations
    inference_args: inference configurations
  """

  predicted_cluster_ids = []
  test_record = []

  train_sequence = train[train.files[0]]
  test_sequence= test[test.files[0]]
  train_cluster_id= train[train.files[1]]
  test_cluster_id= test[test.files[1]]
  #test_cluster_id=test_cluster_id.reshape(-1,1)
  model = UISRNN(args)
  # training
  #model.fit(train_sequence, train_cluster_id, args)
  #model.save(SAVED_MODEL_NAME)
  # we can also skip training by calling：
  model.load(SAVED_MODEL_NAME)
  # testing
  for (test_sequence, test_cluster_id) in zip(test_sequence, test_cluster_id):
    predicted_cluster_id = model.predict(test_sequence, args)
    predicted_cluster_ids.append(predicted_cluster_id)
    print(type(predicted_cluster_id))
    accuracy = compute_sequence_match_accuracy(
        test_cluster_id, predicted_cluster_id)
    test_record.append((accuracy, len(test_cluster_id)))
    print('Ground truth labels:')
    print(test_cluster_id)
    print('Predicted labels:')
    print(predicted_cluster_id)
    print('-' * 80)
  output_string = output_result(args,args,test_record)

  print('Finished diarization experiment')
  print(output_string)
args=arguments()
diarization_experiment(args,train,test)

In [0]:
#results show that if training data is not diverse than trained model cannot perform well on diversed 
#dataset results of using timit-trained model on testing on toy dataset leades to low accuracy on the 
#other side toy-trained model produced 100 % diarization results (because of diversity on train dataset 
#which is not present in timit dataset

In [20]:
import os
os.listdir()

['.config',
 'saved_model_toy.uisrnn',
 'layer_512_1_0.2_result.txt',
 'adc.json',
 'drive',
 'sample_data']